# Notebook de Carga de Datos iniciales

Este notebook esta destinado a la carga inicial de los datos en los distintas base de datos

## Importación de librerias

Se importan las librerias necesarias para la carga de datos masiva.

In [ ]:
# Rutas
from pathlib import Path

#Conexiones BD
from db_connections import client,db_neo4j,db_redis

#Funciones BD
from src import mongo,neo4j,utils,redis
from pymongo.errors import ConnectionFailure

#Procesar archivos/datos
from src.utils import procesar_csv
from datetime import datetime
import os
import pandas as pd


## Limpieza previa

Esta sección elimina toda la información existente en las bases de datos utilizadas por el proyecto, dejándolas listas para una nueva carga o ejecución del proceso completo.
Las acciones que se realizan son las siguientes:

1. **`MongoDB`** → Se elimina la base de datos completa (clase) utilizando `drop_database()`.

2. **`Redis`** → Se vacía completamente la base mediante `flushdb()`.

3. **`Neo4j`** → Se eliminan todos los nodos y relaciones mediante la consulta `MATCH (n) DETACH DELETE n`.

In [ ]:
# --- LIMPIAR DATOS EXISTENTES EN TODAS LAS BASES ---

print("\nLimpiando datos existentes...")

# --- MONGO ---
nombre_base = "clase"
client.drop_database(nombre_base)
print("   ✓ Base de datos Mongo eliminada")

# --- REDIS ---
db_redis.flushdb()
print("   ✓ Base de datos Redis limpiada")

# --- NEO4J ---
with db_neo4j.session() as session:
    session.run("MATCH (n) DETACH DELETE n")
print("   ✓ Nodos y relaciones de Neo4j eliminados")

print("\n Todos los datos fueron limpiados correctamente.")


## Creación de DataSets

Función para crear los distintos DataSets en la carpeta **/fuentes**

In [ ]:
utils.generar_csv_datos_ficticios()

## Carga de Datos

Bloque para la carga masiva de datos de las tres bases de datos

In [ ]:

#-------------------------
# Archivos y colecciones
#-------------------------
nombre_base = "clase"
nombre_colecciones = ["usuarios", "destinos", "hoteles", "reservas", "actividades"]
nombre_archivos = ["usuarios.csv", "destinos.csv", "hoteles.csv", "reservas.csv", "actividades.csv"]

# -------------------------------------------------------------
# PROCESAMIENTO PRINCIPAL
# -------------------------------------------------------------
for nombre_archivo, nombre_coleccion in zip(nombre_archivos, nombre_colecciones):
    df = procesar_csv(nombre_archivo)
    if df is None:
        continue

    mongo.insertar_en_mongo(nombre_base, nombre_coleccion, df)
    redis.insertar_en_redis(nombre_coleccion, df)
    neo4j.crear_nodos_neo4j(nombre_coleccion, df)

    if nombre_coleccion == "reservas":
        neo4j.crear_relaciones_visito(df)

# -------------------------------------------------------------
# RELACIONES ENTRE USUARIOS
# -------------------------------------------------------------
neo4j.crear_relaciones_usuarios()

print("\n🚀 Proceso completado correctamente.")

## Verificación de la carga

Se generan bloques de consultas de prueba por cada una de las bases de datos para comprobar que contengan los datos cargados anteriormente.

### MongoDB

##### Cantidad de documentos existentes en cada colección

In [ ]:
for i in range (len(nombre_colecciones)):
    cantidad=mongo.contar_documentos(nombre_base,nombre_colecciones[i])
    print(f"La cantidad de documentos de la coleccion {nombre_colecciones[i]} es {cantidad}")

### Neo4j

In [ ]:
df_nodos = neo4j.consulta(db_neo4j, """
    MATCH (n)
    RETURN labels(n)[0] AS tipo_nodo, COUNT(*) AS cantidad
""")

df_relaciones = neo4j.consulta(db_neo4j, """
    MATCH ()-[r]->()
    RETURN type(r) AS tipo_relacion, COUNT(*) AS cantidad
""")

print("Nodos por tipo:")
print(df_nodos.to_string(index=False))

print("\nRelaciones por tipo:")
print(df_relaciones.to_string(index=False))


### Redis

##### Cantidad y Vista Previa de Usuarios Conectados

In [ ]:
claves = db_redis.keys("usuario:*:sesion")
print(f"Cantidad de usuarios conectados {len(claves)}\n")
print("Se imprimen los primeros 5:")
for clave in claves[:5]:
    usuario_id = clave.split(":")[1]
    estado = db_redis.get(clave)
    tiempo_restante = db_redis.ttl(clave)
    print(f"Usuario {usuario_id} → sesión: {estado} | TTL: {tiempo_restante} segundos")

##### Cantidad y Vista Previa de Reservas en Carrito

In [ ]:
claves = db_redis.keys("reserva_temp:*")
print(f"Cantidad de reservas temporales {len(claves)}\n")
print("Se imprimen las primeras 5:")
for clave in claves[:5]:
    datos = db_redis.hgetall(clave)
    tiempo_restante = db_redis.ttl(clave)
    print(f"{clave}: {datos} | TTL: {tiempo_restante} segundos")

## Generamos algunas búsquedas en caché

A partir de los datos cargados en MongoDB, se generan consultas ficticias para cargar un caché de búsqueda en Redis.

In [ ]:
#Borrar redis
#db_redis.flushdb()


consultas = [
    ("hoteles", {"ciudad": "La Plata"}),
    ("destinos", {"tipo": "Playa", "ciudad": "La Rioja"}),
    ("hoteles", {"nombre": "Familia Galiano S.L. Hotel"}),
    ("actividades", {"tipo": "relax"})
]

for tipo, filtro in consultas:
    resultado = redis.obtener_cache(tipo, filtro)
    if resultado is None:
        cursor = mongo.obtener_cursor("clase",nombre_coleccion=tipo,filtro=filtro,proyeccion={"_id":0})
        resultado = list(cursor)
        if resultado:
            redis.guardar_en_cache(tipo,filtro,resultado,ttl=10)   

print("✅ Caché de búsqueda creado")

### Vista del caché de búsqueda para lo guardado

A partir de las búsquedas cargadas anteriormente se puede ejecutar el siguiente bloque de código para comprobar que se hayan cargado. El tiempo de expiración seteado de ejemplo es de 10 segundos, por lo cual si se espera ese tiempo, se va a notar como las mismas se borran de Redis.

In [ ]:
for tipo, filtro in consultas:
    resultados = redis.obtener_cache(tipo, filtro)
    if resultados:
        for resultado in resultados:
            print(resultado)
    else:
        print(f"No existe información en caché para la búsqueda {filtro} en {tipo}")